# Neural network determination of crystallite size and microstrain from real XRD data
A. Boulle and A. Souesme, 2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tensorflow import keras
from utilities import Log10Layer
%matplotlib ipympl

### Load data and plot

In [ ]:
# Load data
data = np.load("./data.npz")
data = data["exp_data"][:, :, np.newaxis, np.newaxis]

# Angular range (for plotting only)
tth = np.linspace(5.8438, 27.0799, 10501, endpoint=False)

# Plot
fig, ax = plt.subplots(figsize=(6, 4))
offset = 2
colors = plt.cm.viridis(np.linspace(0, 1, data.shape[0]))
for i in range(data.shape[0]):
    intensity = data[i, :, 0, 0]
    ax.semilogy(tth, intensity + offset * i, color=colors[i], linewidth=0.5)

ax.set_xlabel("2θ (degrees)")
ax.set_ylabel("Intensity (Arb. units)")
plt.show()

### Load trained CNN and run predictions

In [ ]:
# Load CNN
cnn = keras.models.load_model("CNN.hdf5", custom_objects={"Log10Layer": Log10Layer})

# Load normalization scheme
with open("./normL.class", "rb") as f:
    normL = pickle.load(f)

# Run predictions and denormalize
preds = normL.backwards(cnn.predict(data))

### Predicted size and microstrain evolution as a function of time at 1200°C

In [ ]:
time = np.arange(1, preds.shape[0] + 1) * 5.8  # 5.8 minutes between each measurement
fig, [ax0, ax1] = plt.subplots(1, 2, figsize=(8, 4))
ax0.plot(time, preds[:, 0] * 1000, "o")
ax0.set_xlabel("Time (min)")
ax0.set_ylabel(r"Size MgAl$_2$O$_4$ (nm)")
ax1.plot(time, preds[:, 1] * 1e-4, "o")
ax1.set_xlabel("Time (min)")
ax1.set_ylabel(r"Strain MgAl$_2$O$_4$ (%)")
plt.tight_layout()
plt.show()